# F1 Points Predictor

In [2]:
import sqlite3
import pandas as pd
import scipy.stats as ss
import numpy as np
from matplotlib import pyplot as plt

In [3]:
START_YEAR = 2010

CIRCUITS = [
    "bahrain",
    "jeddah",
    "albert_park",
    "baku",
    "imola",
    "monaco",
    "catalunya",
    "villeneuve",
    "red_bull_ring",
    "silverstone",
    "hungaroring",
    "spa",
    "suzuka",
    "monza",
    "marina_bay",
    "americas",
    "rodriguez",
    "interlagos",
    "yas_marina"
]

DRIVERS = [
    "max_verstappen",
    "perez",
    "hamilton",
    "sainz",
    "alonso",
    "norris",
    "leclerc",
    "russell",
    "stroll",
    "gasly",
    "ocon",
    "albon",
    "tsunoda",
    "bottas",
    "hulkenberg",
    "ricciardo",
    "zhou",
    "kevin_magnussen",
    "vettel",
    "raikkonen",
    "grosjean"
]

circuits_csv = ",".join(f"\"{c}\"" for c in CIRCUITS)
drivers_csv = ",".join(f"\"{d}\"" for d in DRIVERS)

In [4]:
con = sqlite3.connect("f1.sql")
cur = con.cursor()

## Data Organization & Cleanup

Get all results from `START_YEAR` to present for the given `CIRCUITS` & `DRIVERS`

In [5]:
circuits = pd.read_sql(
    f"SELECT circuitId, name FROM circuits WHERE circuitRef IN ({circuits_csv})",
    con
)

circuits

,circuitId,name
0,1,Albert Park Grand Prix Circuit
1,3,Bahrain International Circuit
2,4,Circuit de Barcelona-Catalunya
3,6,Circuit de Monaco
4,7,Circuit Gilles Villeneuve
5,9,Silverstone Circuit
6,11,Hungaroring
7,13,Circuit de Spa-Francorchamps
8,14,Autodromo Nazionale di Monza
9,15,Marina Bay Street Circuit


In [6]:
drivers = pd.read_sql_query(
    f"SELECT driverId, code FROM drivers WHERE driverRef in ({drivers_csv})",
    con
)

drivers

,driverId,code
0,1,HAM
1,4,ALO
2,8,RAI
3,20,VET
4,154,GRO
5,807,HUL
6,815,PER
7,817,RIC
8,822,BOT
9,825,MAG


In [7]:
circuit_ids = ",".join(f"{cId}" for cId in circuits.circuitId)
driver_ids = ",".join(f"{dId}" for dId in drivers.driverId)

results = pd.read_sql_query(
    f"""
    SELECT
        races.raceId,
        races.name as race,
        year,
        circuits.name as circuit,
        constructors.name as constructor,
        drivers.code as driver,
        grid as qualifyingPos, 
        position,
        points
    FROM 
        results
        INNER JOIN
            races
            ON races.raceId = results.raceId
        INNER JOIN
            circuits
            ON circuits.circuitId = races.circuitId
        INNER JOIN
            constructors
            ON constructors.constructorId = results.constructorId
        INNER JOIN
            drivers
            ON drivers.driverId = results.driverId
        
    WHERE
        year >= {START_YEAR} AND
        results.driverId IN ({driver_ids}) AND
        circuits.circuitId IN ({circuit_ids})
    
    """, con)

results

,raceId,race,year,circuit,constructor,driver,qualifyingPos,position,points
0,337,Bahrain Grand Prix,2010,Bahrain International Circuit,Ferrari,ALO,3,1.0,25.0
1,337,Bahrain Grand Prix,2010,Bahrain International Circuit,McLaren,HAM,4,3.0,15.0
2,337,Bahrain Grand Prix,2010,Bahrain International Circuit,Red Bull,VET,1,4.0,12.0
3,337,Bahrain Grand Prix,2010,Bahrain International Circuit,Williams,HUL,13,14.0,0.0
4,338,Australian Grand Prix,2010,Albert Park Grand Prix Circuit,Ferrari,ALO,3,4.0,12.0
...,...,...,...,...,...,...,...,...,...
2749,1117,Mexico City Grand Prix,2023,Autódromo Hermanos Rodríguez,Alfa Romeo,BOT,9,15.0,0.0
2750,1117,Mexico City Grand Prix,2023,Autódromo Hermanos Rodríguez,Aston Martin,STR,0,17.0,0.0
2751,1117,Mexico City Grand Prix,2023,Autódromo Hermanos Rodríguez,Aston Martin,ALO,13,NaN,0.0
2752,1117,Mexico City Grand Prix,2023,Autódromo Hermanos Rodríguez,Haas F1 Team,MAG,16,NaN,0.0


### Fix constructors who have merged

In [8]:
results.constructor.value_counts()

Ferrari           377
Red Bull          359
Mercedes          290
McLaren           277
Haas F1 Team      192
Williams          187
Force India       178
Toro Rosso        130
Renault           130
Alfa Romeo        114
AlphaTauri        102
Aston Martin      102
Alpine F1 Team    102
Lotus F1           84
Racing Point       58
Sauber             56
HRT                 8
Manor Marussia      8
Name: constructor, dtype: int64

- Force India, Racing Point -> Aston Martin
- Toro Rosso -> AlphaTauri
- Renault, Lotus F1 -> Alpine F1 Team
- Sauber -> Alfa Romeo

HRT, Manor are defunct now so remove them

In [9]:
constructor_mergers = {
    "Aston Martin": ["Force India", "Racing Point"],
    "AlphaTauri": ["Toro Rosso"],
    "Alpine F1 Team": ["Renault", "Lotus F1"],
    "Alfa Romeo": ["Sauber"]
}

constructor_defunct = ["HRT", "Manor Marussia"]

In [10]:
for present_name, past_names in constructor_mergers.items():
    results.loc[results.constructor.isin(past_names), "constructor"] = present_name

results.drop(results[results.constructor.isin(constructor_defunct)].index, inplace=True)

results.constructor.value_counts()

Ferrari           377
Red Bull          359
Aston Martin      338
Alpine F1 Team    316
Mercedes          290
McLaren           277
AlphaTauri        232
Haas F1 Team      192
Williams          187
Alfa Romeo        170
Name: constructor, dtype: int64

### Adding teammate column

In [11]:
teammates = []
for i in range(results.shape[0]):
    race_id = results.iloc[i].raceId
    constructor = results.iloc[i].constructor
    driver = results.iloc[i].driver
    
    other_driver = results[(results.raceId == race_id) & (results.constructor == constructor) & (results.driver != driver)]
    teammates.append(other_driver.driver.iloc[0] if not other_driver.empty else "OTHER")

results["teammate"] = teammates

### Fix undefined data points

Failed to qualify = 30

DNF on race = 30

In [12]:
# results.qualifyingPos = results.qualifyingPos.replace(0, 30)
results.position = results.position.replace(np.nan, 0).astype(np.int32)

In [13]:
results.to_csv("f1_clean.csv", index=False)

## Calculating DNF rate per circuit

In [14]:
def get_percent_dnf(df):
    n_results = df.shape[0]
    n_dnf = (df.position == 30).sum()
    p_dnf = n_dnf / n_results
    
    return p_dnf

dnf_circuit = results.groupby("circuit").apply(get_percent_dnf).reset_index(name="percentDnf")
dnf_circuit

,circuit,percentDnf
0,Albert Park Grand Prix Circuit,0.250000
1,Autodromo Enzo e Dino Ferrari,0.176471
2,Autodromo Nazionale di Monza,0.135294
3,Autódromo Hermanos Rodríguez,0.138211
4,Autódromo José Carlos Pace,0.136691
5,Bahrain International Circuit,0.131148
6,Baku City Circuit,0.209091
7,Circuit Gilles Villeneuve,0.107914
8,Circuit de Barcelona-Catalunya,0.115607
9,Circuit de Monaco,0.154839


In [15]:
dnf_circuit.to_csv("f1_dnf_circuit.csv", index=False)